# rnn

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# 设置随机种子以确保可重复性
torch.manual_seed(42)
np.random.seed(42)

# 创建RNN模型类
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, hidden):
        # x shape: (batch_size, sequence_length, input_size)
        output, hidden = self.rnn(x, hidden)
        # 只使用最后一个时间步的输出进行预测
        output = self.fc(output[:, -1, :])
        return output, hidden
    
    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size)

# 生成示例数据：简单的正弦波
def generate_sine_wave():
    x = np.linspace(0, 100, 1000)
    y = np.sin(x * 0.1)
    return y

# 准备序列数据
def prepare_sequences(data, seq_length):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        target = data[i + seq_length]
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

# 主函数
def main():
    # 生成数据
    data = generate_sine_wave()
    print(data.shape)
    
    # 参数设置
    sequence_length = 20
    input_size = 1
    hidden_size = 32
    output_size = 1
    batch_size = 32
    num_epochs = 100
    learning_rate = 0.01
    
    # 准备数据
    X, y = prepare_sequences(data, sequence_length)
    train_size = int(len(X) * 0.8)
    
    # 转换为PyTorch张量
    X_train = torch.FloatTensor(X[:train_size]).reshape(-1, sequence_length, 1)
    y_train = torch.FloatTensor(y[:train_size])
    X_test = torch.FloatTensor(X[train_size:]).reshape(-1, sequence_length, 1)
    y_test = torch.FloatTensor(y[train_size:])
    
    # 创建模型
    model = SimpleRNN(input_size, hidden_size, output_size)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # 训练模型
    for epoch in range(num_epochs):
        model.train()
        hidden = model.init_hidden(batch_size)
        
        # 按批次训练
        for i in range(0, len(X_train), batch_size):
            batch_X = X_train[i:i+batch_size]
            batch_y = y_train[i:i+batch_size]
            
            if len(batch_X) < batch_size:
                hidden = model.init_hidden(len(batch_X))
            
            optimizer.zero_grad()
            output, hidden = model(batch_X, hidden.detach())
            loss = criterion(output.squeeze(), batch_y)
            loss.backward()
            optimizer.step()
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    
    # 测试模型
    model.eval()
    with torch.no_grad():
        hidden = model.init_hidden(len(X_test))
        test_output, _ = model(X_test, hidden)
        test_loss = criterion(test_output.squeeze(), y_test)
        print(f'Test Loss: {test_loss.item():.4f}')
        
        # 绘制预测结果
        plt.figure(figsize=(12, 6))
        plt.plot(y_test.numpy(), label='Actual')
        plt.plot(test_output.squeeze().numpy(), label='Predicted')
        plt.legend()
        plt.title('RNN Prediction vs Actual')
        plt.savefig('rnn_prediction.png')
        plt.close()

if __name__ == "__main__":
    main()

(1000,)
Epoch [10/100], Loss: 0.0599
Epoch [20/100], Loss: 0.0840
Epoch [30/100], Loss: 0.0332
Epoch [40/100], Loss: 0.3876
Epoch [50/100], Loss: 0.0007
Epoch [60/100], Loss: 0.0004
Epoch [70/100], Loss: 0.0039
Epoch [80/100], Loss: 0.0012
Epoch [90/100], Loss: 0.0000
Epoch [100/100], Loss: 0.0004
Test Loss: 0.0009


In [5]:
import numpy as np
output1 = np.array([
    [
        [1, 2, 3, 4],
        [5, 6, 7, 8],
        [9, 10, 11, 12]
    ],
    [
        [13, 14, 15, 16],
        [17, 18, 19, 20],
        [21, 22, 23, 24]
    ]
])
print(output1[:,-1,-1])

[12 24]
